# Classifier

## Import libraries

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
from tqdm.notebook import tqdm, trange
from time import sleep
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from collections import Counter
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import winsound


In [32]:
seed = 23
save_model = True
load_model = False

models_directory = "models"
metrics_directory = "metrics"
metrics_name = "metrics_filtered_early.csv"
dataset_name = "Fer2013_merge_selected_85_uniform_aug_surprise_100"
train_df_path = f"../datasets/{dataset_name}/train"
test_df_path = f"../datasets/{dataset_name}/test"

batch_size = 64

In [33]:
if not os.path.isdir(models_directory):
    os.makedirs(name = models_directory)
models_directory_data = os.path.join(models_directory,dataset_name)
if not os.path.isdir(models_directory_data):
    os.makedirs(name = models_directory_data)

## Functions definition

In [34]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [35]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, patience, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD, scheduler_func=None, fold = None):
    history = []
    optimizer = opt_func(model.parameters(), lr, betas=(0.5,0.999))
    scheduler = scheduler_func(optimizer, mode='min', patience=3)
    min_val_loss = sys.float_info.max
    patience_cnt = 0
    for epoch in tqdm(range(epochs), desc = "Current Epoch"):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in tqdm(train_loader, desc = f"Epoch: {epoch}", leave= False):
            optimizer.zero_grad()
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()

        scheduler.step(result['val_loss'])
        result['lrate'] = get_lr(optimizer)

        if epoch > 0 and min_val_loss - result['val_loss'] >= 1e-4:
            patience_cnt = 0
        else:
            patience_cnt += 1
            
        if result['val_loss'] <= min_val_loss:
            min_val_loss = result['val_loss']
            min_val_epoch = epoch
            model_path = os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")
            torch.save(model.state_dict(), model_path)

        if patience_cnt >= patience:
            print('Early stopping')
            break

        model.epoch_end(epoch, result)
        history.append(result)
    model.load_state_dict(torch.load(os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")))
    return history

## Class definition

In [36]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [37]:
class Metric():
    def __init__(self, name):
        self.name = name
    
    def eval(self, outputs, labels):
        pass

In [38]:
class Accuracy(Metric):
    
    def eval(self, outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [39]:
class ImageClassificationBase(nn.Module):
    
    def __init__(self, loss_function, metrics):
        super().__init__()
        self.loss_function = loss_function
        self.metrics = metrics
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = self.loss_function(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = self.loss_function(out, labels)   # Calculate loss
        result = {'val_loss': loss.detach()}
        
        for m in self.metrics:
            result[m.name] = m.eval(out, labels)           # Calculate metrics
            
        return result
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        
        result = {'val_loss': epoch_loss.item()}
        
        for m in self.metrics:
            batch = [x[m.name] for x in outputs]
            epoch = torch.stack(batch).mean()      # Combine metrics
            result[m.name] = epoch.item()
            
        return result
    
    def epoch_end(self, epoch, result):
        out = f"Epoch [{epoch}]"
        vals = list(result.keys())
        for v in vals:
            out += f", {v}: {result[v]:.3e}"
        print(out)
        

In [40]:
class Net(ImageClassificationBase):
    
    def __init__(self, loss_function, metrics, out_size):
        super().__init__(loss_function, metrics)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.norm3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(in_features=128*5*5, out_features=256)
        #self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc = nn.Linear(256, out_size)

    def forward(self, input):
        output = self.norm1(self.pool1(F.relu(self.conv1(input))))
        output = self.norm2(self.pool2(F.relu(self.conv2((output)))))
        output = self.norm3(self.pool3(F.relu(self.conv3((output)))))
        output = output.view(-1, 128*5*5)
        output = F.relu(self.fc1(output))
        #output = F.relu(self.fc2(output))
        output = self.fc(output)
        output = F.softmax(input = output, dim=-1)
        return output

In [41]:
device = get_default_device()
print(device)

cuda


## Dataset loading

In [42]:
df_train = ImageFolder(root=train_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))
df_test = ImageFolder(root=test_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))

In [43]:
train_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size),device)
test_dl = DeviceDataLoader(DataLoader(df_test, batch_size=batch_size, shuffle=False),device)

In [44]:
classes = df_train.classes
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [45]:
counts = dict(Counter(df_train.targets))
print(counts)
weights = np.array(list(counts.values()))
weights = torch.Tensor( min(weights)/weights)
print(weights)

{0: 4110, 1: 365, 2: 4252, 3: 7540, 4: 5168, 5: 5065, 6: 3409}
tensor([0.0888, 1.0000, 0.0858, 0.0484, 0.0706, 0.0721, 0.1071])


## Model design and Training

In [46]:
loss_function = nn.CrossEntropyLoss(weight = weights)
metrics = []
optimizer = optim.Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau

lr = 0.0001
num_epochs = 30
patience = 5

In [47]:
splits = 4
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle=True)
histories = []

for fold,(train_idx,val_idx) in enumerate(skf.split(df_train,train_dl.dl.dataset.targets)):
    print('------------fold no---------{}----------------------'.format(fold))
    
    net = Net(loss_function, metrics, len(classes))
    net.to(device)
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=train_subsampler),device)
    val_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=val_subsampler),device)
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[train_subsampler.indices],return_counts=True))
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[test_subsampler.indices],return_counts=True))
    #print(train_dl[train_subsampler.indices])
    history = dict()
    history["losses"] = fit(num_epochs, patience, lr, net, train_fold_dl, val_fold_dl, optimizer, scheduler, fold)
    history["model"] = net
    histories.append(history)

summary(net,(1,48,48))

------------fold no---------0----------------------


Current Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0:   0%|          | 0/351 [00:00<?, ?it/s]

Epoch [0], val_loss: 1.706e+00, train_loss: 1.784e+00, lrate: 1.000e-04


Epoch: 1:   0%|          | 0/351 [00:00<?, ?it/s]

...

Epoch [20], val_loss: 1.618e+00, train_loss: 1.317e+00, lrate: 1.000e-04


Epoch: 21:   0%|          | 0/351 [00:00<?, ?it/s]

Epoch [21], val_loss: 1.618e+00, train_loss: 1.312e+00, lrate: 1.000e-05


Epoch: 22:   0%|          | 0/351 [00:00<?, ?it/s]

Early stopping
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 46, 46]             320
         MaxPool2d-2           [-1, 32, 23, 23]               0
       BatchNorm2d-3           [-1, 32, 23, 23]              64
            Conv2d-4           [-1, 64, 21, 21]          18,496
         MaxPool2d-5           [-1, 64, 11, 11]               0
       BatchNorm2d-6           [-1, 64, 11, 11]             128
            Conv2d-7            [-1, 128, 9, 9]          73,856
         MaxPool2d-8            [-1, 128, 5, 5]               0
       BatchNorm2d-9            [-1, 128, 5, 5]             256
           Linear-10                  [-1, 256]         819,456
           Linear-11                    [-1, 7]           1,799
Total params: 914,375
Trainable params: 914,375
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB)

## Save/Load model

In [48]:
if save_model:
    for fold,h in enumerate(histories):
        model_path = os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")
        torch.save(h["model"].state_dict(), model_path)

In [49]:
if load_model:
    histories = []

    for fold, model in enumerate(os.listdir(models_directory_data)):
        net = Net(loss_function, metrics, len(classes))
        net.load_state_dict(torch.load(os.path.join(models_directory_data,model)))
        net.eval()
        net.cuda()
        histories.append( {"model": net } )

## Model testing

In [50]:
def get_model_predicitons(model, dataset):
    y_test = []
    y_scores = []

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in tqdm(dataset):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)

            y_test.extend(labels.tolist())
            y_scores.extend(outputs.tolist())

    y_test = np.array(y_test)
    y_scores = np.array(y_scores)

    return y_test, y_scores


### Network Accuracy

In [51]:
def evaluate_accuracy(y_test, y_scores, classes, verbose=False):
    # Overall accuracy

    n_test = y_test.shape[0]
    predicted = np.argmax(y_scores,axis=1)
    mean_acc = np.sum(predicted == y_test) / n_test
    class_acc = []
    
    # Accuracy per class

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # collect the correct predictions for each class
    for label, prediction in zip(y_test, predicted):
        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1

    for classname, correct_count in correct_pred.items():
        accuracy = float(correct_count) / total_pred[classname]
        class_acc.append(accuracy)

    if verbose:
        print(f'Overall accuracy: {100 * mean_acc } %')

        print("Accuracy per class:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {class_acc[i] * 100:.1f} %')
        
    return mean_acc, class_acc

# ROC/AUC

In [52]:

def evaluate_roc_auc(y_test, y_scores, classes, plot=False, verbose=False):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    ths = dict()
    roc_auc = dict()
    for i,_ in enumerate(classes):
        fpr[i], tpr[i], ths[i] = roc_curve(y_test == i, y_scores[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    if plot:
        # Plot of a ROC curve for a specific class
        for i,c in enumerate(classes):
            plt.figure()
            plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC {c}')
            plt.legend(loc="lower right")
            plt.show()

    mean_auc = np.mean(list(roc_auc.values()))

    if verbose:
        print(f"Mean AUC: { mean_auc * 100} %")

        print("Per classs AUC:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {roc_auc[i]*100:.2f} %')

    return mean_auc, roc_auc

In [53]:
y_test = dict()
y_scores = dict()

for fold,h in enumerate(histories):
    y_test[fold], y_scores[fold] = get_model_predicitons(h["model"],test_dl)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [54]:
columns=["mean AUC", "angry AUC", "disgust AUC", "fear AUC", "happy AUC", "neautral AUC", "sad AUC", "surprise AUC", "mean ACC", "angry ACC", "disgust ACC", "fear ACC", "happy ACC", "neautral ACC", "sad ACC", "surprise ACC"]
df = pd.DataFrame(columns = columns)

for fold,_ in enumerate(histories):
    mean_acc,class_acc = evaluate_accuracy(y_test[fold], y_scores[fold], classes)
    mean_auc,roc_auc = evaluate_roc_auc(y_test[fold], y_scores[fold], classes)

    exp = [mean_auc]
    exp += list(roc_auc.values())
    exp += [mean_acc]
    exp += class_acc

    df_new = pd.DataFrame( np.array([exp]),[f"{dataset_name} fold {fold}"], columns=columns)
    df = pd.concat([df,df_new])
    df = df.astype('float')

mean_df = df.mean(0).to_frame().transpose().set_index(pd.Index([f"{dataset_name} mean"]))
df = pd.concat([df,mean_df])
df


,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_merge_selected_85_uniform_aug_surprise_100 fold 0,0.854021,0.789183,0.916983,0.755017,0.922250,0.863250,0.810900,0.920567,0.561429,0.500,0.5100,0.4200,0.730,0.6000,0.3800,0.790
Fer2013_merge_selected_85_uniform_aug_surprise_100 fold 1,0.854896,0.814450,0.914017,0.734783,0.922117,0.856917,0.825667,0.916325,0.564286,0.510,0.5200,0.2700,0.760,0.5900,0.5100,0.790
Fer2013_merge_selected_85_uniform_aug_surprise_100 fold 2,0.849142,0.790867,0.906950,0.753817,0.905325,0.875183,0.792550,0.919300,0.555714,0.500,0.5700,0.3900,0.770,0.5600,0.3500,0.750
Fer2013_merge_selected_85_uniform_aug_surprise_100 fold 3,0.844864,0.826517,0.908467,0.716817,0.920508,0.850450,0.792967,0.898325,0.550000,0.470,0.4900,0.3300,0.760,0.5800,0.4500,0.770
Fer2013_merge_selected_85_uniform_aug_surprise_100 mean,0.850731,0.805254,0.911604,0.740108,0.917550,0.861450,0.805521,0.913629,0.557857,0.495,0.5225,0.3525,0.755,0.5825,0.4225,0.775


## Save metrics

In [55]:
try:
    stored_df = pd.read_csv(os.path.join(metrics_directory, metrics_name),index_col=0)
except FileNotFoundError:
    stored_df = pd.DataFrame(columns = columns)
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.851951,0.802933,0.897417,0.746271,0.921156,0.866823,0.820529,0.908527,0.565714,0.4850,0.5150,0.4175,0.7225,0.6025,0.4625,0.7550
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.851527,0.810425,0.901646,0.748487,0.920058,0.867377,0.814950,0.897748,0.558214,0.4825,0.5000,0.3975,0.7375,0.5750,0.4625,0.7525
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.852117,0.820917,0.898250,0.751125,0.923150,0.865158,0.803012,0.903208,0.566429,0.5175,0.4900,0.4350,0.7225,0.6100,0.4425,0.7475
Fer2013_merge_selected_85_uniform xval-4,0.850587,0.811165,0.903354,0.750200,0.915410,0.864867,0.812550,0.896560,0.565357,0.5175,0.5050,0.3875,0.7200,0.5975,0.4750,0.7550
Fer2013_merge_selected_85_uniform_aug_disgust_5000 xval-4,0.855798,0.822617,0.886883,0.749129,0.920315,0.867692,0.833283,0.910665,0.567500,0.4725,0.4875,0.4525,0.7150,0.6225,0.4750,0.7475
Fer2013_merge_selected_85_uniform_aug_surprise xval-4,0.850800,0.814192,0.906971,0.737621,0.921637,0.864408,0.806792,0.903977,0.558571,0.4875,0.5025,0.3500,0.7300,0.6000,0.4675,0.7725
Fer2013_merge_selected_85_uniform_aug_surprise xval-4,0.849465,0.809892,0.895792,0.741225,0.923658,0.856435,0.818304,0.900946,0.565357,0.5100,0.5175,0.3625,0.7200,0.5950,0.4875,0.7650
Fer2013_merge_selected_85_uniform_aug_surprise_500 xval-4,0.846230,0.802542,0.905979,0.728835,0.907238,0.861575,0.812129,0.905310,0.554643,0.4675,0.5200,0.3650,0.7050,0.6125,0.4675,0.7450


In [56]:
stored_df = pd.concat([stored_df,mean_df.set_index(pd.Index([f"{dataset_name} xval-{splits}"]))])
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.851951,0.802933,0.897417,0.746271,0.921156,0.866823,0.820529,0.908527,0.565714,0.4850,0.5150,0.4175,0.7225,0.6025,0.4625,0.7550
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.851527,0.810425,0.901646,0.748487,0.920058,0.867377,0.814950,0.897748,0.558214,0.4825,0.5000,0.3975,0.7375,0.5750,0.4625,0.7525
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.852117,0.820917,0.898250,0.751125,0.923150,0.865158,0.803012,0.903208,0.566429,0.5175,0.4900,0.4350,0.7225,0.6100,0.4425,0.7475
Fer2013_merge_selected_85_uniform xval-4,0.850587,0.811165,0.903354,0.750200,0.915410,0.864867,0.812550,0.896560,0.565357,0.5175,0.5050,0.3875,0.7200,0.5975,0.4750,0.7550
Fer2013_merge_selected_85_uniform_aug_disgust_5000 xval-4,0.855798,0.822617,0.886883,0.749129,0.920315,0.867692,0.833283,0.910665,0.567500,0.4725,0.4875,0.4525,0.7150,0.6225,0.4750,0.7475
Fer2013_merge_selected_85_uniform_aug_surprise xval-4,0.850800,0.814192,0.906971,0.737621,0.921637,0.864408,0.806792,0.903977,0.558571,0.4875,0.5025,0.3500,0.7300,0.6000,0.4675,0.7725
Fer2013_merge_selected_85_uniform_aug_surprise xval-4,0.849465,0.809892,0.895792,0.741225,0.923658,0.856435,0.818304,0.900946,0.565357,0.5100,0.5175,0.3625,0.7200,0.5950,0.4875,0.7650
Fer2013_merge_selected_85_uniform_aug_surprise_500 xval-4,0.846230,0.802542,0.905979,0.728835,0.907238,0.861575,0.812129,0.905310,0.554643,0.4675,0.5200,0.3650,0.7050,0.6125,0.4675,0.7450
Fer2013_merge_selected_85_uniform_aug_surprise_100 xval-4,0.850731,0.805254,0.911604,0.740108,0.917550,0.861450,0.805521,0.913629,0.557857,0.4950,0.5225,0.3525,0.7550,0.5825,0.4225,0.7750


In [57]:
if not os.path.isdir(metrics_directory):
    os.makedirs(name = metrics_directory)
stored_df.to_csv(os.path.join(metrics_directory, metrics_name))

In [58]:
stored_df.index.name = "Method"
stored_df.groupby("Method").mean()

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Method,,,,,,,,,,,,,,,,
Fer2013_merge_selected_85_uniform xval-4,0.850587,0.811165,0.903354,0.750200,0.915410,0.864867,0.812550,0.896560,0.565357,0.51750,0.505000,0.387500,0.7200,0.597500,0.475000,0.755000
Fer2013_merge_selected_85_uniform_aug_disgust xval-4,0.851865,0.811425,0.899104,0.748628,0.921455,0.866453,0.812831,0.903161,0.563452,0.49500,0.501667,0.416667,0.7275,0.595833,0.455833,0.751667
Fer2013_merge_selected_85_uniform_aug_disgust_5000 xval-4,0.855798,0.822617,0.886883,0.749129,0.920315,0.867692,0.833283,0.910665,0.567500,0.47250,0.487500,0.452500,0.7150,0.622500,0.475000,0.747500
Fer2013_merge_selected_85_uniform_aug_surprise xval-4,0.850132,0.812042,0.901381,0.739423,0.922648,0.860422,0.812548,0.902461,0.561964,0.49875,0.510000,0.356250,0.7250,0.597500,0.477500,0.768750
Fer2013_merge_selected_85_uniform_aug_surprise_100 xval-4,0.850731,0.805254,0.911604,0.740108,0.917550,0.861450,0.805521,0.913629,0.557857,0.49500,0.522500,0.352500,0.7550,0.582500,0.422500,0.775000
Fer2013_merge_selected_85_uniform_aug_surprise_500 xval-4,0.846230,0.802542,0.905979,0.728835,0.907238,0.861575,0.812129,0.905310,0.554643,0.46750,0.520000,0.365000,0.7050,0.612500,0.467500,0.745000


In [59]:
winsound.Beep(400, 2000)

In [60]:
winsound.Beep(400, 2000)